# Proof of concept
## Get Groundstate of LiH with VQE, AdaptVqe and StatefulVQE
- taken from: https://qiskit-community.github.io/qiskit-nature/howtos/adapt_vqe.html

In [37]:
# imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import time
from datetime import datetime

## Define problem with ansatz

In [38]:
#driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
#driver = PySCFDriver(atom="C 0 0 -0.6025; H 0 0 -1.6691; C 0 0 0.6025; H 0 0 1.6691", basis="sto-3g")
#driver = PySCFDriver(atom="O 0 0 0; H  0 1 0; H 0 0 1", basis="sto-3g")
driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5", basis="sto-3g")
#driver = PySCFDriver(atom="He 0 0 0; He 0 0 0.9", basis="sto-3g")
problem = driver.run()

In [39]:
mapper = JordanWignerMapper()

In [40]:
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [41]:
print(problem.num_spatial_orbitals, problem.num_particles, ansatz.width())

6 (2, 2) 12


In [42]:
# define vqe
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

## implement StatefulVQE
- taken over from qiskit-nature-cp2k

In [43]:
# imports
import logging

from enum import Enum

from qiskit.quantum_info.operators.base_operator import BaseOperator
from qiskit.circuit.library import EvolvedOperatorAnsatz

from qiskit_algorithms.list_or_dict import ListOrDict
from qiskit_algorithms import VQEResult

from qiskit_algorithms.observables_evaluator import estimate_observables
from qiskit_algorithms.utils import validate_bounds, validate_initial_point
from qiskit_algorithms.utils.set_batching import _set_default_batchsize

In [44]:
logger = logging.getLogger(__name__)

In [45]:
def depth_filter(param):
    inst, qubits, clbits = param
    return inst.num_qubits == 2

In [46]:
class StatefulVQE(VQE):
    def compute_minimum_eigenvalue(
        self,
        operator: BaseOperator,
        aux_operators: ListOrDict[BaseOperator] | None = None,
    ) -> VQEResult:
        if self.ansatz.num_parameters == 0:
            eigenvalue = estimate_observables(self.estimator, self.ansatz, [operator])[0][0]

            optimizer_result = OptimizerResult()
            optimizer_result.x = []
            optimizer_result.fun = eigenvalue
            optimizer_result.jac = None
            optimizer_result.nfev = 0
            optimizer_result.njev = 0
            optimizer_result.nit = 0

            optimizer_time = 0

            if aux_operators is not None:
                aux_operators_evaluated = estimate_observables(
                    self.estimator,
                    self.ansatz,
                    aux_operators,
                    optimizer_result.x,
                )
            else:
                aux_operators_evaluated = None

            return self._build_vqe_result(
                self.ansatz,
                optimizer_result,
                aux_operators_evaluated,
                optimizer_time,
            )

        self._check_operator_ansatz(operator)

        initial_point = validate_initial_point(self.initial_point, self.ansatz)

        bounds = validate_bounds(self.ansatz)

        start_time = time.time()

        evaluate_energy = self._get_evaluate_energy(self.ansatz, operator)

        if self.gradient is not None:
            evaluate_gradient = self._get_evaluate_gradient(self.ansatz, operator)
        else:
            evaluate_gradient = None

        # perform optimization
        if callable(self.optimizer):
            optimizer_result = self.optimizer(
                fun=evaluate_energy,
                x0=initial_point,
                jac=evaluate_gradient,
                bounds=bounds,
            )
        else:
            # we always want to submit as many estimations per job as possible for minimal
            # overhead on the hardware
            was_updated = _set_default_batchsize(self.optimizer)

            optimizer_result = self.optimizer.minimize(
                fun=evaluate_energy,
                x0=initial_point,
                jac=evaluate_gradient,
                bounds=bounds,
            )

            # reset to original value
            if was_updated:
                self.optimizer.set_max_evals_grouped(None)

        optimizer_time = time.time() - start_time

        logger.info(
            "Optimization complete in %s seconds.\nFound optimal point %s",
            optimizer_time,
            optimizer_result.x,
        )

        # stateful aspect to permit warm-starting of the algorithm
        self.initial_point = optimizer_result.x

        if aux_operators is not None:
            aux_operators_evaluated = estimate_observables(
                self.estimator, self.ansatz, aux_operators, optimizer_result.x
            )
        else:
            aux_operators_evaluated = None

        decomposed = self.ansatz.decompose().decompose().decompose()
        logger.info(f"The circuit has the following gates: %s", str(decomposed.count_ops()))
        logger.info(f"The circuit has a depth of %s", str(decomposed.depth()))
        logger.info(f"The circuit has a 2-qubit gate depth of %s", str(decomposed.depth(depth_filter)))
        logger.info(f"The circuit has %s paramters", str(decomposed.num_parameters))

        return self._build_vqe_result(
            self.ansatz, optimizer_result, aux_operators_evaluated, optimizer_time
        )

## now setup StatefulAdaptVQE

In [47]:
stateful_vqe = StatefulVQE(Estimator(), ansatz, SLSQP())
stateful_vqe.initial_point = np.zeros(ansatz.num_parameters)

## Now get some results for VQE

In [48]:
solver = GroundStateEigensolver(mapper, vqe)

In [49]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Code execution time [sec]: 3653.9124372005463
Total ground state energy = -7.8824


## Now get some results for AdaptVQE

In [50]:
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

In [51]:
solver = GroundStateEigensolver(mapper, adapt_vqe)

In [52]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Code execution time [sec]: 1551.8469829559326
Total ground state energy = -7.8820


## Now get some results for StatefulVQE

In [53]:
solver = GroundStateEigensolver(mapper, stateful_vqe)

In [54]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Code execution time [sec]: 3133.5566153526306
Total ground state energy = -7.8824
